In [1]:
import os, sys
import pandas as pd
from Bio.Seq import reverse_complement, translate
from sPAMUtils import dict_pam_disrup_rank

In [8]:
df = pd.read_parquet('./tests/test_data/ABL1_ex4_pos189A_G_output.parquet')
df

,ID,WT74_On,Edited74_On,PBSlen,RTlen,RT-PBSlen,Edit_pos,Edit_len,RHA_len,type_sub,...,nGCcnt1,nGCcnt2,nGCcnt3,fGCcont1,fGCcont2,fGCcont3,MFE3,MFE4,DeepSpCas9_score,PE2max_score
0,ABL1_ex4_pos189A_G,GGTGTGTCCCCCAACTACGACAAGTGGGAGATGGAACGCACGGACA...,xxxxxxxxxxxxxxxTACGACAAGTGGGAGATGGAACGCACGGACA...,6,35,41,34,1,1,1,...,3,19,22,50.000000,54.285714,53.658537,-5.3,-0.6,48.486732,0.870687
1,ABL1_ex4_pos189A_G,GGTGTGTCCCCCAACTACGACAAGTGGGAGATGGAACGCACGGACA...,xxxxxxxxxxxxxxCTACGACAAGTGGGAGATGGAACGCACGGACA...,7,35,42,34,1,1,1,...,4,19,23,57.142857,54.285714,54.761905,-5.3,-0.6,48.486732,5.386128
2,ABL1_ex4_pos189A_G,GGTGTGTCCCCCAACTACGACAAGTGGGAGATGGAACGCACGGACA...,xxxxxxxxxxxxxACTACGACAAGTGGGAGATGGAACGCACGGACA...,8,35,43,34,1,1,1,...,4,19,23,50.000000,54.285714,53.488372,-5.3,-0.6,48.486732,4.485147
3,ABL1_ex4_pos189A_G,GGTGTGTCCCCCAACTACGACAAGTGGGAGATGGAACGCACGGACA...,xxxxxxxxxxxxAACTACGACAAGTGGGAGATGGAACGCACGGACA...,9,35,44,34,1,1,1,...,4,19,23,44.444444,54.285714,52.272727,-5.3,-0.6,48.486732,3.391637
4,ABL1_ex4_pos189A_G,GGTGTGTCCCCCAACTACGACAAGTGGGAGATGGAACGCACGGACA...,xxxxxxxxxxxCAACTACGACAAGTGGGAGATGGAACGCACGGACA...,10,35,45,34,1,1,1,...,5,19,24,50.000000,54.285714,53.333333,-5.3,-0.6,48.486732,4.873809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,ABL1_ex4_pos189A_G,ACGCCCTCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,xxxxxxxxGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,13,27,40,26,1,1,1,...,9,19,28,69.230769,70.370370,70.000000,-12.0,-2.0,28.402525,12.416884
1208,ABL1_ex4_pos189A_G,ACGCCCTCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,xxxxxxxCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,14,27,41,26,1,1,1,...,10,19,29,71.428571,70.370370,70.731707,-11.2,-2.0,28.402525,12.165054
1209,ABL1_ex4_pos189A_G,ACGCCCTCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,xxxxxxTCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,15,27,42,26,1,1,1,...,10,19,29,66.666667,70.370370,69.047619,-11.2,-2.0,28.402525,10.777733
1210,ABL1_ex4_pos189A_G,ACGCCCTCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,xxxxxCTCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...,16,27,43,26,1,1,1,...,11,19,30,68.750000,70.370370,69.767442,-11.2,-2.0,28.402523,8.894163


In [9]:
# test_info = df.iloc[362]  # (+) strand target
test_info = df.iloc[1048] # (-) strand target
print(test_info)
print(test_info.WT74_On)
print(test_info.Edited74_On)

ID                                                 ABL1_ex4_pos189A_G
WT74_On             ACGCCCTCGTACACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...
Edited74_On         xxxxxxxxxxxCACCTCCCCGTACTGGCCCCCGCCCAGCTTGTGCT...
PBSlen                                                             10
RTlen                                                              40
RT-PBSlen                                                          50
Edit_pos                                                           26
Edit_len                                                            1
RHA_len                                                            14
type_sub                                                            1
type_ins                                                            0
type_del                                                            0
Tm1                                                         42.558042
Tm2                                                         73.394542
Tm2new              

In [10]:
ref_seq   = 'GTCTATGGTGTGTCCCCCAACTACGACAAGTGGGAGATGGAACGCACGGACATCACCATGAAGCACAAGCTGGGCGGGGGCCAGTACGGGGAGGTGTACGAGGGCGTGTGGAAGAAATACA'
frame     = 0
cds_start = 0
cds_end   = 121

In [21]:
class SynonymousPAM:

    def __init__(self,
                 dp_record:pd.Series,
                 ref_seq:str,
                 frame:int,
                 cds_start:int,
                 cds_end:int,
                 adj_rha:bool=True,
                 ):
        """DeepPrime output으로 만들어진 파일에서 pegRNA들에 silent mutation을 함께 유발하는 것 중 최적의 pegRNA를 선택해주는 것
        모든 기능은 prime editing으로 1bp substitution을 했을 때를 가정하여 만들어졌다.
        우선, intended edit 기준으로 best pegRNA들을 선정한다.
        그 후, 아래와 같이 silent PAM co-editing을 추가한다.

        Case 1: Silent PAM co-editing이 LHA 부분에 발생하는 경우,
        그대로 silent PAM co-editing을 넣고 완성

        Case 2: Silent PAM co-editing이 RHA 부분에 발생하는 경우,
        원래 pegRNA의 RHA 길이를 기준으로 맞춰주기 위해 RTT 길이를 늘려주기.
        
        Case 3: Intended edit이 이미 PAM 위치 (+5 or +6)인 경우,
        1) 만약 남은 PAM 위치 (G)에 silent mutation이 가능하면 그 것으로 결정
        2) 남은 PAM 위치의 SNV 중 silent mutation이 없다면, LHA에서 가능한 silent mutation을 넣어준다.

        Args:
            dp_record (pd.Series): DeepPrime을 돌리고 output으로 나오는 것의 한 index의 정보를 가져온 것
            ref_seq (str): 해당 mutation을 만들었을 떄, 사용한 reference sequence (121nt)
            frame (int): Ref_seq의 frame을 표시. 실제 계산할 때에는 Edit position 기준으로 amino acid frame을 표시 (0, 1, 2)
            cds_start (int): CDS가 시작되는 위치, 그 이전 위치에서부터는 silent mutation을 만들 수 없으므로 고르는 위치에서 제외
            cds_end (int): CDS가 종료되는 위치, 그 이후 위치에서부터는 silent mutation을 만들 수 없으므로 고르는 위치에서 제외
            adj_rha (bool, optional): silent mutation이 RHA에 위치하는 경우, 기존의 pegRNA에서의 RHA 길이만큼을 유지하기 위해 RTT를 늘려주는 기능. 기본값은 실행.

        Raises:
            ValueError: frame이 0, 1, 2 중에 하나로 입력되지 않은 경우
            ValueError: Reference sequence가 pegRNA와 matching이 안 될 경우
        """        
    
        # input error check
        if frame not in [0, 1, 2]: raise ValueError('Frame should be 0, 1 or 2')

        # step 1: 각종 서열 정보들을 가져온다.
        self.sID      = dp_record.ID
        self.wt_seq   = dp_record.WT74_On
        self.pbs_dna  = dp_record.Edited74_On.replace('x', '')[:dp_record.PBSlen]
        self.rtt_dna  = dp_record.Edited74_On.replace('x', '')[-dp_record.RTlen:]
        self.edit_pos = dp_record.Edit_pos

        # ============ fwd / rev 각각에 따라서 함수 만들고 각각에 맞는 pegRNA로 변환하는 함수  ============
        
        if self.wt_seq in ref_seq:
            self.strand = '+'
            self.rtt_frame = (3 - ((self.edit_pos-1) % 3) + (3 - frame)) % 3 # rtt 시작점의 frame, LHA 길이를 이용한 계산
            self.output = self.make_synonyPAM_RTT_fwd(self.rtt_frame, self.rtt_dna, self.strand) # edit position / Silent mut 고려해서 뽑은 것들
            
            # 만약 edit pos가 이미 PAM을 targeting 하고 있는 등의 이유로, 뽑히는 PAM silent Mut이 없다면, LHA에서 뽑기
            if len(self.output) == 0: print('Get silent Mut at LHA region!')

        elif reverse_complement(self.wt_seq) in ref_seq:
            self.strand = '-'
            self.rtt_frame = (self.edit_pos + (3 - frame)) % 3  # revcom_rtt_dna의 3' end가 위치하는 지점의 frame. 시작점이 거기이기 때문.
            self.output = self.make_synonyPAM_RTT_rev(self.rtt_frame, self.rtt_dna, self.strand) # edit position / Silent mut 고려해서 뽑은 것들

            # 만약 edit pos가 이미 PAM을 targeting 하고 있는 등의 이유로, 뽑히는 PAM silent Mut이 없다면, LHA에서 뽑기
            if len(self.output) == 0: print('Get silent Mut at LHA region!')
            
        else: raise ValueError('Reference sequence is not matched with pegRNA information!\nPlease chech your ref_seq')
        
        
        
        # 여기서 만들어진 mutation 중, 가장 rank가 높은 것을 선택해서 바꿔주기
        # 기존 RTT_DNA에서 rtt_frame == 1 인 경우에만 rtt_dna[2:5]를 바꿔준다.
        # else: rtt_dna[3:6]을 바꿔준다. 
        

        # if adj_rha == True:
            # if edit_pos >= 5:
                # PAM 위치가 LHA에 존재하는 경우 또는 prime editing 자체가 PAM editing을 하는 경우
                # Synonymous mutation을 LHA 영역에서 고르면 된다. 
    
    # End def __init__:
        

        
    def make_snv(self, seq:str, pos:int) -> list:
        """sequence에서 지정된 위치에 A/T/G/C로 만들어진 SNV를 만들어서 list로 return하는 함수

        Args:
            seq (str): SNV를 만들 서열 정보
            pos (int): seq에서 SNV를 만들 위치 정보

        Returns:
            list: 특정 위치에 가능한 SNV들이 모인 list
        """        

        list_base = ['A', 'G', 'C', 'T']
        list_sSNV = []

        for base in list_base:
            list_sSeq = list(seq)
            list_sSeq[pos] = base
            snv_temp = ''.join(list_sSeq)
            if seq!=snv_temp: list_sSNV.append(snv_temp)

        return list_sSNV
    # def END: make_snv
    
    def make_synonyLHA_fwd(self, ) -> pd.DataFrame:
        """만약 이미 pegRNA의 intended edit position이 +5 or +6인 경우,
        silent mutation은 LHA에 만들어주는 함수
        그 중, strand 방향이 (+)인 경우에 대한 함수이다. 

        Returns:
            pd.DataFrame: _description_
        """        
        
        print('Intended edit pos is: %s' % self.edit_pos)
        
    # def End: make_synonyLHA_fwd
        
    
    
    def make_synonyPAM_RTT_fwd(self, rtt_frame:int, rtt_dna:str, strand:str) -> pd.DataFrame:
        """pegRNA가 CDS strand 방향과 동일한 경우 (+)
        PAM sequence (GG)에서 frame에 따라 가능한 synonymous mutation들을 만들고,
        이에 대한 결과를 DataFrame으로 만들어준다. 

        Args:
            rtt_frame (int): CDS에서 RTT의 frame을 의미함 (0, 1, 2).
            rtt_dna (str): pegRNA의 RTT 부분을 DNA로 가져온 것. cDNA와 동일.
            strand (str): Reference sequence 기준으로 pegRNA의 방향 (+ / -)

        Returns:
            pd.DataFrame: PAM 위치에 가능한 synonymous mutation 정보들
        """   

        if   rtt_frame == 0: dict_codon_pamPos = {rtt_dna[3:6]: [1, 2]}
        elif rtt_frame == 1: dict_codon_pamPos = {rtt_dna[2:5]: [2]} # codon 맨 앞의 G가 바뀌었을 때에는 silent mut이 없음
        else               : dict_codon_pamPos = {rtt_dna[3:6]: [0, 1]}
    
        dict_pam_edit = {
            'Codon_WT'      : [],
            'Codon_Mut'     : [],
            'RTT_DNA_Mut'  : [],
            'RTT_DNA_frame' : [],
            'RTT_DNA_Strand': [],
            'AminoAcid_WT'  : [],
            'AminoAcid_Mut' : [],
            'Silent_check'  : [],
            'PAM_Mut'       : [],
            'PAM_edit_rank' : [],
        }

        PAM_G_pos = 5
        
        # for loop: GG PAM 위치가 걸쳐있는 codon들을 가져온다.
        for codon in dict_codon_pamPos:
            # for loop: 각 codon들에서 GG PAM에 해당하는 위치들을 가져온다.
            for pam_pos in dict_codon_pamPos[codon]:
                
                list_mut_codon = self.make_snv(codon, pam_pos)
                
                for mut_codon in list_mut_codon:
                    
                    aa_wt  = translate(codon)
                    aa_mut = translate(mut_codon)
                    
                    dict_pam_edit['Codon_WT'].append(codon)
                    dict_pam_edit['Codon_Mut'].append(mut_codon)
                    dict_pam_edit['RTT_DNA_frame'].append(rtt_frame)
                    dict_pam_edit['RTT_DNA_Strand'].append(strand)
                    dict_pam_edit['AminoAcid_WT'].append(aa_wt)
                    dict_pam_edit['AminoAcid_Mut'].append(aa_mut)
                    dict_pam_edit['Silent_check'].append(aa_wt==aa_mut)
                    
                    if PAM_G_pos == 5: pam_mut = mut_codon[pam_pos] + 'G'
                    else             : pam_mut = 'G' + mut_codon[pam_pos]
                        
                    dict_pam_edit['PAM_G_pos'].append(PAM_G_pos)
                    dict_pam_edit['PAM_Mut'].append(pam_mut)
                    dict_pam_edit['PAM_edit_rank'].append(dict_pam_disrup_rank[pam_mut])

                    rtt_dna_mut = self.rtt_dna[:4] + pam_mut + self.rtt_dna[6:]

                    dict_pam_edit['RTT_DNA_Mut'].append(rtt_dna_mut)
                    
            PAM_G_pos += 6
                    
        self.pam_mut  = pd.DataFrame(dict_pam_edit) 
        df_synonymous = self.pam_mut.groupby(by='Silent_check') 
        
        return df_synonymous.get_group(True)
    # def END: make_synonymousPAM_RTT_fwd
    
    def make_synonyPAM_RTT_rev(self, rtt_frame:int, rtt_dna:str, strand:str) -> pd.DataFrame:
        """pegRNA가 CDS strand 방향과 반대인 경우 (-)
        우선 RTT_DNA sequence를 reverse complementary sequence로 가져온 다음
        PAM sequence (CC)에서 frame에 따라 가능한 synonymous mutation들을 만들고,
        이에 대한 결과를 DataFrame으로 만들어준다. 

        Args:
            rtt_frame (int): CDS에서 RTT의 frame을 의미함 (0, 1, 2).
            rtt_dna (str): pegRNA의 RTT 부분을 DNA로 가져온 것. cDNA와 동일.
            strand (str): Reference sequence 기준으로 pegRNA의 방향 (+ / -)

        Returns:
            pd.DataFrame: PAM 위치에 가능한 synonymous mutation 정보들
        """        

        # strand가 (-)인 경우에는 RT-PBS를 revcom으로 바꿔서 PAM 기준으로 SNV들을 만들기
        
        rc_rtt_dna = reverse_complement(rtt_dna)
        
        if   rtt_frame == 0: dict_codon_pamPos = {rc_rtt_dna[-6:-3]: [0, 1]}
        elif rtt_frame == 1: dict_codon_pamPos = {rc_rtt_dna[-7:-4]: [1, 2]}
        else               : dict_codon_pamPos = {rc_rtt_dna[-8:-5]: [2], rc_rtt_dna[-5:-2]: [0]}
        
        dict_pam_edit = {
            'Codon_WT'      : [],
            'Codon_Mut'     : [],
            'RTT_DNA_Mut'  : [],
            'RTT_DNA_frame' : [],
            'RTT_DNA_Strand': [],
            'AminoAcid_WT'  : [],
            'AminoAcid_Mut' : [],
            'Silent_check'  : [],
            'PAM_G_pos'     : [],
            'PAM_Mut'       : [],
            'PAM_edit_rank' : [],
        }

        PAM_G_pos = 6
        
        # for loop: GG PAM 위치가 걸쳐있는 codon들을 가져온다.
        for codon in dict_codon_pamPos:
            # for loop: 각 codon들에서 GG PAM에 해당하는 위치들을 가져온다.
            for pam_pos in dict_codon_pamPos[codon]:
                
                list_mut_codon = self.make_snv(codon, pam_pos)
                
                for mut_codon in list_mut_codon:
                    
                    aa_wt  = translate(codon)
                    aa_mut = translate(mut_codon)
                    
                    dict_pam_edit['Codon_WT'].append(codon)
                    dict_pam_edit['Codon_Mut'].append(mut_codon)
                    dict_pam_edit['RTT_DNA_frame'].append(rtt_frame)
                    dict_pam_edit['RTT_DNA_Strand'].append(strand)
                    dict_pam_edit['AminoAcid_WT'].append(aa_wt)
                    dict_pam_edit['AminoAcid_Mut'].append(aa_mut)
                    dict_pam_edit['Silent_check'].append(aa_wt==aa_mut)
                    
                    if PAM_G_pos == 6: pam_mut = 'G' + reverse_complement(mut_codon[pam_pos])
                    else             : pam_mut = reverse_complement(mut_codon[pam_pos]) + 'G'
                        
                    dict_pam_edit['PAM_G_pos'].append(PAM_G_pos)
                    dict_pam_edit['PAM_Mut'].append(pam_mut)
                    dict_pam_edit['PAM_edit_rank'].append(dict_pam_disrup_rank[pam_mut])
                    
                    rtt_dna_mut = self.rtt_dna[:4] + pam_mut + self.rtt_dna[6:]

                    dict_pam_edit['RTT_DNA_Mut'].append(rtt_dna_mut)
                    
            PAM_G_pos -= 1

        self.pam_mut  = pd.DataFrame(dict_pam_edit) 
        df_synonymous = self.pam_mut.groupby(by='Silent_check')
        
        return df_synonymous.get_group(True)
    # def END: make_synonymousPAM_RTT_rev



df_out = SynonymousPAM(test_info, ref_seq=ref_seq, frame=frame, cds_start=cds_start, cds_end=cds_end)
df_out.output



,Codon_WT,Codon_Mut,RTT_DNA_Mut,RTT_DNA_frame,RTT_DNA_Strand,AminoAcid_WT,AminoAcid_Mut,Silent_check,PAM_G_pos,PAM_Mut,PAM_edit_rank
0,GGC,GGA,TACTGTCCCCCGCCCAGCTTGTGCTCCATGGTGATGTCCG,2,-,G,G,True,6,GT,9
1,GGC,GGG,TACTGCCCCCCGCCCAGCTTGTGCTCCATGGTGATGTCCG,2,-,G,G,True,6,GC,14
2,GGC,GGT,TACTGACCCCCGCCCAGCTTGTGCTCCATGGTGATGTCCG,2,-,G,G,True,6,GA,13


In [16]:
df_out.rtt_dna

'TACTGGCCCCCGCCCAGCTTGTGCTCCATGGTGATGTCCG'

In [13]:
df_out.pam_mut

,Codon_WT,Codon_Mut,RTT_DNA_frame,RTT_DNA_Strand,AminoAcid_WT,AminoAcid_Mut,Silent_check,PAM_Mut,PAM_edit_rank
0,GGC,GGA,2,-,G,G,True,GT,9
1,GGC,GGG,2,-,G,G,True,GC,14
2,GGC,GGT,2,-,G,G,True,GA,13
3,CAG,AAG,2,-,Q,K,False,TG,5
4,CAG,GAG,2,-,Q,E,False,CG,15
5,CAG,TAG,2,-,Q,*,False,AG,3
